# 添加所需要的库

In [1]:
import syft as sy
from syft.workers.websocket_client import WebsocketClientWorker
from src.websocket_client import MyWebsocketClientWorker
from src.nn_model import ConvNet1D, loss_fn
from src.my_utils import generate_kwarg
import torch
from datetime import datetime
import asyncio

In [2]:
client_device_mapping_id = {
    "192.168.3.5": "AA",
    "192.168.3.6": "BB",
    "192.168.3.9": "CC",
    "192.168.3.15": "DD",
    "192.168.3.16": "EE",
}

hook = sy.TorchHook(torch)

# 连接当前确认的中心节点
aa_kwarg = generate_kwarg('AA')
node_aa = MyWebsocketClientWorker(hook=hook, **aa_kwarg)

# 在聚合初始化模型
command = {"command_name": "model_initialization"}
node_aa.command(command)

Websocket connection closed (worker: AA)
Created new websocket connection


RuntimeError: Websocket connection closed and creation of new connection failed.

In [ ]:
# 分发第一部分模型
command = {
    "command_name": "model_dissemination",
    "forward_device_mapping_id": {
        "192.168.3.6": "BB",
        "192.168.3.9": "CC",
    }
}
node_aa.command(command)

In [ ]:
# 分发第二次模型
command_1 = {
    "command_name": "model_dissemination",
    "forward_device_mapping_id": {
        "192.168.3.15": "DD"
    }
}

command_2 = {
    "command_name": "model_dissemination",
    "forward_device_mapping_id": {
        "192.168.3.16": "EE"
    }
}

cmds = [command_1, command_2]

bb_kwarg = generate_kwarg('BB')
cc_kwarg = generate_kwarg('CC')
all_nodes = []
for kwarg in [bb_kwarg, cc_kwarg]:
    all_nodes.append(MyWebsocketClientWorker(hook=hook, **kwarg))

async def main():
    await asyncio.gather(
        *[
            n.async_command(cmd)
            for n, cmd in zip(all_nodes, cmds)
        ]
    )

In [ ]:
await main()